In [63]:
import os
import sys
import numpy as np
import deepdish as dd
import pandas as pd
from argparse import Namespace

In [7]:
from gwpopulation_pipe.data_collection import load_gwtc_1_events



args = Namespace(gwtc1_samples_regex='/home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW*.hdf5')

posteriors = load_gwtc_1_events(args)


16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW150914_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW151012_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW151226_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170104_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170608_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170729_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170809_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170814_GWTC-1.hdf5.
16:53 bi

NameError: name 'postertior' is not defined

In [9]:
posteriors['/home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW150914_GWTC-1.hdf5']

,redshift,mass_1,a_1,cos_tilt_1,mass_2,a_2,cos_tilt_2,mass_ratio
0,0.108213,35.225515,0.417147,-0.280624,33.427289,0.867740,0.403853,0.948951
1,0.085328,31.898266,0.125709,-0.757349,31.496839,0.260679,-0.312285,0.987415
2,0.078865,35.124252,0.581047,0.649781,31.487262,0.926893,-0.510843,0.896454
3,0.082373,33.641806,0.235808,0.116578,32.968738,0.094391,-0.720505,0.979993
4,0.073895,36.760824,0.511521,-0.438237,29.467517,0.868009,0.269333,0.801601
...,...,...,...,...,...,...,...,...
8345,0.066089,35.233424,0.484003,0.194507,31.288003,0.627191,-0.408345,0.888020
8346,0.097492,34.464289,0.589654,-0.737792,33.416243,0.650758,0.875384,0.969590
8347,0.094773,34.768191,0.708407,0.852085,32.662388,0.714805,-0.797475,0.939433
8348,0.116737,40.192456,0.389284,-0.251461,28.289953,0.521304,0.830526,0.703862


In [64]:
import pandas as pd
import glob
import tqdm
import re

SAMPLE_SRC = "/home/avi.vajpeyi/projects/agn_phenomenological_model/data/gwtc_samples/"
PARAM_TO_KEEP = ["a_1", "a_2", "cos_tilt_1", "cos_tilt_2", "mass_ratio", "mass_1", "mass_2", "redshift", "cos_theta_12", "chi_p", "chi_eff"]

posterior_files = [f for f in glob.glob(SAMPLE_SRC+"*.csv") if "prior" not in f]
prior_files = glob.glob(SAMPLE_SRC + "*_prior.csv")

reject_events = ["GW170817", "S190425z", "S190426c", "S190719an", "190814", "S190909w", "S190930ak"]
posts =[]
events =[]
def get_event_name(fname):
    return re.findall(r"(\w*\d{6}[a-z]*)", fname)[0]

def add_agn_samples_to_df(df):
    df['s1x'], df['s1y'], df['s1z'] = df['spin_1x'], df['spin_1y'], df['spin_1z']
    df['s2x'], df['s2y'], df['s2z'] = df['spin_2x'], df['spin_2y'], df['spin_2z']
    df['s1_dot_s2'] = (df['s1x'] * df['s2x']) + (df['s1y'] * df['s2y']) + (
            df['s1z'] * df['s2z'])
    df['s1_mag'] = np.sqrt(
        (df['s1x'] * df['s1x']) + (df['s1y'] * df['s1y']) + (df['s1z'] * df['s1z']))
    df['s2_mag'] = np.sqrt(
        (df['s2x'] * df['s2x']) + (df['s2y'] * df['s2y']) + (df['s2z'] * df['s2z']))
    df['cos_theta_12'] = df['s1_dot_s2'] / (df['s1_mag'] * df['s2_mag'])
    return df


def read_posteriors(posterior_files):
    # posterior_files = [f for f in glob.glob(SAMPLE_SRC+"*.csv") if "prior" not in f]
    posteriors = {}
    for f in tqdm.tqdm(posterior_files, desc='Posteriors'):
        event = get_event_name(f)
        if event not in reject_events:
            d = pd.read_csv(f,sep=" ", index_col=False)
            try:
                d = add_agn_samples_to_df(d)
                posteriors.update({f:d[PARAM_TO_KEEP]})
            except Exception as e:
                print(f"Skipping {event}: {e}\n{d.columns.values.tolist()}")
    return posteriors



    
posteriors =  read_posteriors(posterior_files)
# priors = {f:pd.read_csv(f,sep=" ", index_col=False) for f in tqdm.tqdm(prior_files, desc="Priors")}


agn_samples = "/home/avi.vajpeyi/projects/agn_phenomenological_model/data/gwtc_agn_samples/" 
os.mkdir(agn_samples)

for fname, posterior in posteriors.items():
    new_f = os.path.join(agn_samples, os.path.basename(fname))
    posterior.to_csv(new_f, index=False, sep=' ')

Posteriors:  30%|██▉       | 26/88 [00:06<00:13,  4.57it/s]

Skipping GW170729: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors:  48%|████▊     | 42/88 [00:10<00:10,  4.18it/s]

Skipping GW170814: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors:  53%|█████▎    | 47/88 [00:10<00:07,  5.78it/s]

...

Posteriors:  89%|████████▊ | 78/88 [00:16<00:01,  7.97it/s]

Skipping GW170818: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']
Skipping GW170809: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors:  95%|█████████▌| 84/88 [00:17<00:00,  9.58it/s]

Skipping GW170608: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors: 100%|██████████| 88/88 [00:17<00:00,  4.92it/s]


In [62]:
from copy import deepcopy
import matplotlib
matplotlib.rcParams["text.usetex"] = False  # noqa
import matplotlib.pyplot as plt

for filename in posteriors.keys():
    event = get_event_name(filename)
    if event not in reject_events:
        posts.append(posteriors[filename])
        events.append(event)

        
def plot_summary(posteriors: list, events: list):
    posteriors = deepcopy(posteriors)
    plot_dir = os.path.join("outdir/data")
    plot_chi = True
    for posterior in posteriors:
        if all(
            [
                key in posterior
                for key in ["a_1", "a_2", "cos_tilt_1", "cos_tilt_2", "mass_ratio"]
            ]
        ):
            posterior["chi_eff"] = (
                posterior["a_1"] * posterior["cos_tilt_1"]
                + posterior["a_2"] * posterior["cos_tilt_2"] * posterior["mass_ratio"]
            ) / (1 + posterior["mass_ratio"])
            posterior["chi_p"] = np.maximum(
                posterior["a_1"] * (1 - posterior["cos_tilt_1"] ** 2) ** 0.5,
                posterior["mass_ratio"]
                * (3 + 4 * posterior["mass_ratio"])
                / (4 + 3 * posterior["mass_ratio"])
                * posterior["a_2"]
                * (1 - posterior["cos_tilt_2"] ** 2) ** 0.5,
            )
        else:
            plot_chi = False
    plot_mass_2 = True
    for posterior in posteriors:
        try:
            posterior["mass_2"] = posterior["mass_1"] * posterior["mass_ratio"]
        except KeyError:
            plot_mass_2 = False
    print(f"Plotting params {posteriors[0].keys()}")
    for parameter in posteriors[0].keys():
        if parameter in ["chi_eff", "chi_p"] and not plot_chi:
            continue
        elif parameter == "mass_2" and not plot_mass_2:
            continue
        print(f"Making box plot for {parameter}")
        file_name = os.path.join(plot_dir, parameter)
        data = [post[parameter] for post in posteriors]
        fig = plt.figure(figsize=(len(posteriors), 5))
        plt.violinplot(data)
        plt.ylabel(parameter.replace("_", " "))
        plt.xticks(np.arange(1, len(events) + 1), events, rotation=90)
        if parameter == "prior":
            plt.yscale("log")
        plt.tight_layout()
        plt.savefig(file_name + ".png")
        plt.close(fig)
        
plot_summary(posts, events)

Making box plot for a_1
Making box plot for a_2
Making box plot for cos_tilt_1
Making box plot for cos_tilt_2
Making box plot for mass_ratio
Making box plot for mass_1
Making box plot for mass_2
Making box plot for redshift
Making box plot for cos_theta_12
Making box plot for chi_p
Making box plot for chi_eff
